In [128]:
import numpy as np

In [129]:
def grad_phi(l, w, B):
    
    size = len(w)
    
#     print(np.dot(B, np.ones(size)).A1)
    grad_phi_1 = l - np.dot(B, np.ones(size)).A1 / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
    grad_phi_2 = w - np.dot(B.T, np.ones(size)).A1 / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
    
    return grad_phi_1, grad_phi_2, np.concatenate((grad_phi_1.A1, grad_phi_2.A1))

In [130]:
np.dot(np.array([1,2,3]), np.array([1, 2, 3]))

14

In [131]:
def phi(x, l, w, B):
    
    size = len(w)
    return np.dot(x[:size], l) + np.dot(x[size:], w) + np.log(np.dot(np.dot(np.ones(size).T, B), np.ones(size)))

In [132]:
def f(correspondance_matrix, cost_matrix, gamma):
    
    return np.sum(correspondance_matrix * np.log(correspondance_matrix)) + gamma * np.sum(corresponance_matrix * cost_matrix)

In [133]:
import pandas as pd

In [134]:
df = pd.read_csv('moscow.csv')

In [135]:
df.head()

,area_i,area_j,live_i_work_j,time_ij,dist_ij
0,1,1,40,26,2.2
1,1,2,4,49,8.1
2,1,3,3,35,6.8
3,1,4,2,75,9.3
4,1,5,2,30,8.0


In [136]:
size = df.area_i.max()

In [137]:
df.area_i = df.area_i.astype(int)
df.area_j = df.area_j.astype(int)

In [138]:
times_matrix, distance_matrix, corresponance_matrix = np.zeros((size, size)), np.zeros((size, size)), np.zeros((size, size))

In [139]:
for i in df.index:
    
    area_i, area_j, live_work_ij, time_ij, dist_ij = df.loc[i]
    area_i, area_j = int(area_i) - 1, int(area_j) - 1
    
    times_matrix[area_i, area_j] = time_ij
    distance_matrix[area_i, area_j] = dist_ij
    corresponance_matrix[area_i, area_j] = live_work_ij

In [140]:
L = 1
L_new = None
k = 0
a = 0
# size = 10
beta = 1

# T = np.matrix([np.random.uniform(size=size) for _ in range(size)])
# T /= T.sum(axis=1)

T = 2 * np.matrix(times_matrix ** 4)
T /= T.sum(axis=1)

# l, w = np.random.uniform(0, 1, size), np.random.uniform(0, 1, size)  
l, w = df.groupby('area_i')['live_i_work_j'].sum(), df.groupby('area_j')['live_i_work_j'].sum()
l /= l.sum()
w /= w.sum()

lambdas_l, lambdas_w = np.zeros(size), np.zeros(size)
x, y, v = np.zeros(2 * size), np.zeros(2 * size), np.zeros(2 * size)

d_hat = 1

In [141]:
eps_f = 0.001

In [142]:
while True:
    if L_new is not None:
        L = L_new
        
    L_new = L / 2
    
    while True:
    
        a_new = 1 / (2 * L) + np.sqrt(1 / (4 * L ** 2) + a ** 2 * L / L_new)
        tau = 1 / (a_new * L_new)
        
        y = tau * v + (1 - tau) * x
        
        B = np.exp(-beta * T - lambdas_l - lambdas_w)
        grad_phi_1, grad_phi_2, grad_phi_value = grad_phi(y[:size], y[size:], B)

        lambdas_l, lambdas_w = x[:size].copy(), x[size:].copy()
        
        if np.linalg.norm(grad_phi_1) >= np.linalg.norm(grad_phi_2):
            lambdas_l_new = lambdas_l + np.log(l) - np.log(np.exp((-beta * T.T - lambdas_l).T - lambdas_w).sum(axis=1).A1)
            lambdas_w_new = lambdas_w.copy() 
        else:
            lambdas_l_new = lambdas_l.copy()
            lambdas_w_new = lambdas_w + np.log(w) - np.log(np.exp((-beta * T.T - lambdas_l).T - lambdas_w).sum(axis=0).A1)
        
        x_new = np.concatenate((lambdas_l_new, lambdas_w_new))
        v_new = v - a_new * grad_phi_value
        
        if phi(x_new, l, w, B) <= phi(y, l, w, B) - np.linalg.norm(grad_phi_value) ** 2 / (2 * L_new):
            
            d = B / np.dot(np.dot(np.ones(size).T, B), np.ones(size))
            
            d_hat_new = (a_new * d + L * a ** 2 * d_hat) / (L_new * a_new ** 2)
            break
        
        L_new *= 2
    
    print(abs(f(d_hat_new, T, beta) + phi(x_new, l, w, B)))
    print(np.linalg.norm(np.dot(d_hat_new.T, np.ones(size).T).A1 - l), np.linalg.norm(np.dot(d_hat_new, np.ones(size).T).A1 - w))
    
    if abs(f(d_hat_new, T, beta) + phi(x_new, l, w, B)) < eps_f:
        break
    
    d_hat = d_hat_new.copy()


[[1723.50892492]]
0.2509970171268909 0.27402913374070864
[[1723.50892492]]
0.2509970171268909 0.27402913374070864
[[1723.50892492]]
0.2509970171268909 0.27402913374070864
[[1723.50892492]]
0.2509970171268909 0.27402913374070864
[[1723.50892492]]
0.2509970171268909 0.27402913374070864
[[1723.50892492]]
0.2509970171268909 0.27402913374070864
[[1723.50892492]]
0.2509970171268909 0.27402913374070864
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.17

[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.171762339

[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.171762339

[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.171762339

[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.171762339

[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.171762339

[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.171762339

[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.171762339

[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.1717623398094422
[[1829.15440677]]
0.1288012644027169 0.171762339

KeyboardInterrupt: 

In [125]:
np.dot(d_hat_new.T, np.ones(size)).A1 - l

area_i
1     0.009818
2    -0.018055
3    -0.019131
4    -0.005079
5    -0.012279
6    -0.013959
7    -0.021233
8    -0.016431
9     0.002336
10    0.042779
11   -0.016218
12   -0.079798
13   -0.013659
14   -0.000998
15   -0.016394
16    0.009023
17    0.013173
18    0.013365
19    0.022556
20    0.043708
21    0.040422
22    0.036054
Name: live_i_work_j, dtype: float64

In [105]:
number_of_people = df.groupby('area_j')['live_i_work_j'].sum().sum()

In [106]:
d_restored = d_hat_new * number_of_people

In [109]:
np.sum((d_restored - corresponance_matrix) ** 2) / size ** 2

84.67277664652211

In [108]:
np.sum((d_restored - corresponance_matrix) ** 2) / size ** 2

84.67277664652211